# Imports

In [3]:
import pandas as pd

Initialize functions

In [ ]:
def categorize(df, categories):
    for category in categories:
        df[category] = pd.Categorical(df[category]).codes
    return df

def numberize(df, numerics):
    for numeric in numerics:
        df[numeric] = df[numeric].astype("float")
    return df

# Function to clean year_built values
def clean_year_built(year):
    try:
        # Attempt to convert the value to an integer
        year = int(float(year))  # Handles both integer and float strings
        year_str = str(year)

        # Ensure the string is exactly 4 characters long
        if len(year_str) > 4:
            return int(year_str[:4])
        elif len(year_str) < 4:
            return int(year_str.ljust(4, '0'))  # Pads with zeros if less than 4 digits
        else:
            return year
    except (ValueError, TypeError):
        # Return NaN for non-numeric values or None
        return pd.NA

load and prepare the data

In [ ]:
df = pd.read_csv("flatfox.csv")
df.drop(df[df["offer_type"] != "RENT"].index, inplace=True)
# remove columns with wrong year
df.drop(df[df["year_built"] < 1800].index, inplace=True)
df.drop(df[df["year_built"] > 2024].index, inplace=True)
# NaN  in floors is replaced by 0
# year_built has unrealistic values (for example 19 Million)
#df['year_built'] = df['year_built'].apply(clean_year_built)

df.fillna({"year_built": df["year_built"].mean(), "floor": 0.0, "year_renovated": df["year_built"], "number_of_rooms": 0.0, "livingspace": 0.0}, inplace=True)


df.dropna(subset=["price_display"], inplace=True)
df.drop([
    "pk",
    "slug",
    "url",
    "short_url",
    "moving_date",
    "rent_net",
    "rent_charges",
    "rent_gross",
    "offer_type",
    "submit_url",
    "status",
    "created",
    "reference",
    "ref_property",
    "ref_house",
    "ref_object",
    "alternative_reference",
    "published",
    "short_title",
    "public_title",
    "pitch_title",
    "description_title",
    "description",
    "attributes",
    "public_address",
    "video_url",
    "tour_url",
    "website_url",
    "live_viewing_url",
    "cover_image",
    "images",
    "documents",
    "agency",
    "rent_title",
    "surface_property",
    "surface_living",
    "surface_usable",
    "surface_usable_minimum",
    "volume",
    "space_display",
    "street"], axis=1, inplace=True)

categories = [
    "object_category",
    "object_type",
    "price_display_type",
    "price_unit",
    "city",
    "moving_date_type"
]
numerics = [
    "is_furnished",
    "is_temporary",
    "is_selling_furniture",
    "reserved"
]
categorize(df, categories)
numberize(df, numerics)
df.describe()
#df.head()
#df.isna().sum()
#df.dtypes

count all years: 21040

count 1500-2024: 20725

count 1800-2024: 20672


Write clean data to csv

In [ ]:
df.to_csv("flatfox_clean.csv", index=False)

Create profiling report

In [ ]:
from ydata_profiling import ProfileReport
ProfileReport(pd.read_csv("flatfox.csv", low_memory=False).drop(["created", "published"], axis=1), title="Flatfox Profiling").to_file("flatfox_profiling.html")
ProfileReport(pd.read_csv("flatfox_clean.csv", low_memory=False), title="Flatfox Profiling").to_file("flatfox_clean_profiling.html")